##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Trabajo con tensores dispersos

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/sparse_tensor"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/sparse_tensor_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/sparse_tensor_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/sparse_tensor_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

Cuando se trabaja con tensores que contienen muchos valores cero, es importante almacenarlos de una forma eficiente en cuanto a tiempo y espacio. Los tensores dispersos permiten el almacenamiento y el procesamiento eficiente de tensores que contienen muchos valores cero. Los tensores dispersos se usan ampliamente en esquemas de codificación como [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), como parte del preprocesamiento de datos en aplicaciones de procesamiento de lenguaje natural (NLP) y para el procesamiento previo de imágenes con muchos pixeles oscuros en aplicaciones de visión para computadoras.

## Tensores dispersos en TensorFlow

TensorFlow representa a los tensores dispersos mediante el objeto `tf.sparse.SparseTensor`. Actualmente, los tensores dispersos que se encuentran en TensorFlow están codificados con el formato de lista de coordenadas (COO). Este formato de codificación se optimiza para matrices hiperdispersas como incrustaciones.

La codificación COO para tensores dispersos está compuesta por lo siguiente:

- `values`: un tensor D 1 con forma `[N]` que contiene todos los valores distintos de cero.
- `indices`: un tensor D 2 con forma `[N, rank]` que contiene los índices de los valores distintos de cero.
- `dense_shape`: un tensor D 1 con forma `[rank]`, con el que se especifica la forma del tensor.

Un valor ***nonzero*** (distinto de cero) en el contexto de un `tf.sparse.SparseTensor` es un valor que no está codificado explícitamente. Es posible incluir valores cero explícitamente en los `values` de una matriz dispersa COO, pero estos "ceros explícitos", por lo general, no se incluyen cuando se refieren a valores distintos de cero en un tensor disperso.

Nota: `tf.sparse.SparseTensor` no requiere que los índices o valores se encuentren en un orden en particular, pero para varias operaciones se asume que están en un orden de filas (<em>row-major</em>). Use `tf.sparse.reorder` para crear una copia del tensor disperso que se ordene en el orden canónico de filas. 

## Creación de un `tf.sparse.SparseTensor`

Para construir tensores dispersos, especifique directamente sus `values`, `indices` y `dense_shape`.

In [ ]:
import tensorflow as tf

In [ ]:
st1 = tf.sparse.SparseTensor(indices=[[0, 3], [2, 4]],
                      values=[10, 20],
                      dense_shape=[3, 10])

<img src="images/sparse_tensor.png">

Cuando use la función `print()` para imprimir un tensor disperso, mostrará el contenido de los tres tensores componentes:

In [ ]:
print(st1)

Es más fácil entender el contenido de un tensor disperso si los `values` distintos de cero están alineados con sus correspondientes `indices`. Defina una función ayudante para formatear tensores dispersos con un formato legible, de modo que cada valor distinto de cero se muestre en su propia línea.

In [ ]:
def pprint_sparse_tensor(st):
  s = "<SparseTensor shape=%s \n values={" % (st.dense_shape.numpy().tolist(),)
  for (index, value) in zip(st.indices, st.values):
    s += f"\n  %s: %s" % (index.numpy().tolist(), value.numpy().tolist())
  return s + "}>"

In [ ]:
print(pprint_sparse_tensor(st1))

También se pueden construir tensores dispersos a partir de tensores densos con `tf.sparse.from_dense` y convertirlos usando `tf.sparse.to_dense`.

In [ ]:
st2 = tf.sparse.from_dense([[1, 0, 0, 8], [0, 0, 0, 0], [0, 0, 3, 0]])
print(pprint_sparse_tensor(st2))

In [ ]:
st3 = tf.sparse.to_dense(st2)
print(st3)

## Manipulación de tensores dispersos

Use las utilidades del paquete `tf.sparse` para manipular los tensores dispersos. Las operaciones como `tf.math.add` que se pueden usar para manipulación aritmética de tensores densos no funcionan con tensores dispersos.

Agregue tensores dispersos de la misma forma mediante `tf.sparse.add`. 

In [ ]:
st_a = tf.sparse.SparseTensor(indices=[[0, 2], [3, 4]],
                       values=[31, 2], 
                       dense_shape=[4, 10])

st_b = tf.sparse.SparseTensor(indices=[[0, 2], [7, 0]],
                       values=[56, 38],
                       dense_shape=[4, 10])

st_sum = tf.sparse.add(st_a, st_b)

print(pprint_sparse_tensor(st_sum))

Use `tf.sparse.sparse_dense_matmul` para multiplicar tensores dispersos con matrices densas.

In [ ]:
st_c = tf.sparse.SparseTensor(indices=([0, 1], [1, 0], [1, 1]),
                       values=[13, 15, 17],
                       dense_shape=(2,2))

mb = tf.constant([[4], [6]])
product = tf.sparse.sparse_dense_matmul(st_c, mb)

print(product)

Ponga junto los tensores dispersos con `tf.sparse.concat` y sepárelos con `tf.sparse.slice`.


In [ ]:
sparse_pattern_A = tf.sparse.SparseTensor(indices = [[2,4], [3,3], [3,4], [4,3], [4,4], [5,4]],
                         values = [1,1,1,1,1,1],
                         dense_shape = [8,5])
sparse_pattern_B = tf.sparse.SparseTensor(indices = [[0,2], [1,1], [1,3], [2,0], [2,4], [2,5], [3,5], 
                                              [4,5], [5,0], [5,4], [5,5], [6,1], [6,3], [7,2]],
                         values = [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
                         dense_shape = [8,6])
sparse_pattern_C = tf.sparse.SparseTensor(indices = [[3,0], [4,0]],
                         values = [1,1],
                         dense_shape = [8,6])

sparse_patterns_list = [sparse_pattern_A, sparse_pattern_B, sparse_pattern_C]
sparse_pattern = tf.sparse.concat(axis=1, sp_inputs=sparse_patterns_list)
print(tf.sparse.to_dense(sparse_pattern))

In [ ]:
sparse_slice_A = tf.sparse.slice(sparse_pattern_A, start = [0,0], size = [8,5])
sparse_slice_B = tf.sparse.slice(sparse_pattern_B, start = [0,5], size = [8,6])
sparse_slice_C = tf.sparse.slice(sparse_pattern_C, start = [0,10], size = [8,6])
print(tf.sparse.to_dense(sparse_slice_A))
print(tf.sparse.to_dense(sparse_slice_B))
print(tf.sparse.to_dense(sparse_slice_C))

Si está usando una versión de TensorFlow 2.4 o posterior, use `tf.sparse.map_values` para operaciones de elemento por elemento para valores distintos de cero en tensores dispersos.

In [ ]:
st2_plus_5 = tf.sparse.map_values(tf.add, st2, 5)
print(tf.sparse.to_dense(st2_plus_5))

Tenga en cuenta que solamente los valores distintos de cero se modificaron, los valores cero permanecen en cero.

De un modo equivalente, para versiones anteriores de TensorFlow, se puede seguir el patrón de diseño que se encuentra a continuación.

In [ ]:
st2_plus_5 = tf.sparse.SparseTensor(
    st2.indices,
    st2.values + 5,
    st2.dense_shape)
print(tf.sparse.to_dense(st2_plus_5))

## `tf.sparse.SparseTensor` con otras API de TensorFlow

Los tensores dispersos funcionan con transparencia con estas API de TensorFlow:

- `tf.keras`
- `tf.data`
- `tf.Train.Example` (<em>protobuf</em>)
- `tf.function`
- `tf.while_loop`
- `tf.cond`
- `tf.identity`
- `tf.cast`
- `tf.print`
- `tf.saved_model`
- `tf.io.serialize_sparse`
- `tf.io.serialize_many_sparse`
- `tf.io.deserialize_many_sparse`
- `tf.math.abs`
- `tf.math.negative`
- `tf.math.sign`
- `tf.math.square`
- `tf.math.sqrt`
- `tf.math.erf`
- `tf.math.tanh`
- `tf.math.bessel_i0e`
- `tf.math.bessel_i1e`

A continuación, se muestran ejemplos de algunas de las API mencionadas arriba.

### `tf.keras`

Un subconjunto de las API `tf.keras` admite tensores dispersos sin <em>castings</em> ni conversiones costosas de operaciones. La API Keras permite pasar tensores dispersos como entradas a un modelo Keras. Establezca `sparse=True` cuando llame a `tf.keras.Input` o `tf.keras.layers.InputLayer`. Puede pasar tensores dispersos entre capas de Keras y también hacer que los modelos de Keras los devuelvan como resultado. Si usa tensores dispersos en las capas `tf.keras.layers.Dense` de su modelo, como resultado obtendrá tensores densos.

En el ejemplo a continuación se muestra cómo pasar un tensor disperso como entrada para un modelo Keras si solamente se usan capas que admiten entradas dispersas.

In [ ]:
x = tf.keras.Input(shape=(4,), sparse=True)
y = tf.keras.layers.Dense(4)(x)
model = tf.keras.Model(x, y)

sparse_data = tf.sparse.SparseTensor(
    indices = [(0,0),(0,1),(0,2),
               (4,3),(5,0),(5,1)],
    values = [1,1,1,1,1,1],
    dense_shape = (6,4)
)

model(sparse_data)

model.predict(sparse_data)

### `tf.data`

La API `tf.data` le permite desarrollar canalizaciones de entradas complejas a partir de piezas reutilizables simples. Su estructura de datos principales es `tf.data.Dataset`, que representa una secuencia de elementos en la que cada elemento está compuesto por uno o más componentes.

#### Cómo construir conjuntos de datos con tensores dispersos

Es posible construir conjuntos de datos a partir de tensores dispersos con los mismos métodos que se usan para construirlos a partir de arreglos `tf.Tensor` o NumPy, tales como `tf.data.Dataset.from_tensor_slices`. Esta operación preserva la dispersión (o la naturaleza dispersa) de los datos.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(sparse_data)
for element in dataset: 
  print(pprint_sparse_tensor(element))

#### Cómo agrupar o desagrupar conjuntos de datos con tensores dispersos

Es posible agrupar en lotes (combinar elementos consecutivos para formar un solo elemento) y desagrupar conjuntos de datos con tensores dispersos mediante los métodos `Dataset.batch` y `Dataset.unbatch` respectivamente.

In [ ]:
batched_dataset = dataset.batch(2)
for element in batched_dataset:
  print (pprint_sparse_tensor(element))

In [ ]:
unbatched_dataset = batched_dataset.unbatch()
for element in unbatched_dataset:
  print (pprint_sparse_tensor(element))

También se puede usar `tf.data.experimental.dense_to_sparse_batch` para agrupar en tensores dispersos aquellos elementos de un conjunto de datos que tengan formas diversas. 

#### Cómo transformar conjuntos de datos con tensores dispersos

Transforma y crea tensores dispersos en conjuntos de datos con `Dataset.map`.

In [ ]:
transform_dataset = dataset.map(lambda x: x*2)
for i in transform_dataset:
  print(pprint_sparse_tensor(i))

### tf.train.Example

`tf.train.Example` <br>es una codificación de búfer de protocolo (<em>protobuf</em>) estándar para datos de TensorFlow. Cuando use tensores dispersos con `tf.train.Example`, puede hacer lo siguiente:

- Leer datos de longitudes variables en un `tf.sparse.SparseTensor` con `tf.io.VarLenFeature`. Sin embargo, deberá considerar usar `tf.io.RaggedFeature` en su lugar.

- Leer datos dispersos arbitrarios en un `tf.sparse.SparseTensor` con `tf.io.SparseFeature`, que usa tres claves de funciones separadas para almacenar `indices`, `values` y `dense_shape`.

### `tf.function`

La `tf.function` decoradora calcula previamente los grafos de TensorFlow para funciones de Python, lo que puede mejorar considerablemente el rendimiento de su código de TensorFlow. Los tensores dispersos funcionan de forma transparente tanto con `tf.function` como con [funciones concretas](https://www.tensorflow.org/guide/function#obtaining_concrete_functions).

In [ ]:
@tf.function
def f(x,y):
  return tf.sparse.sparse_dense_matmul(x,y)

a = tf.sparse.SparseTensor(indices=[[0, 3], [2, 4]],
                    values=[15, 25],
                    dense_shape=[3, 10])

b = tf.sparse.to_dense(tf.sparse.transpose(a))

c = f(a,b)

print(c)

## Cómo distinguir valores faltantes de valores cero

La mayoría de la operaciones `tf.sparse.SparseTensor` tratan exactamente del mismo modo a los valores faltantes y a los valores cero explícitos. Nos referimos al diseño, se supone que un `tf.sparse.SparseTensor` se comporte del mismo modo que un tensor denso.

Sin embargo, hay algunos casos en los que puede ser útil distinguir valores cero de valores faltantes. En particular, esta posibilidad permite contar con una manera de codificar datos faltantes/desconocidos en los datos de entrenamiento. Por ejemplo, consideremos un caso de uso en el que hay un tensor de puntaje (que puede tener cualquier valor de punto flotante entre -Inf y +Inf), con algunos puntajes faltantes. En este caso, es posible codificar este tensor con un tensor disperso en el que los ceros explícitos sean puntajes cero conocidos, pero que los valores cero implícitos, en realidad, representen los datos faltantes y no cero.

Nota: Por lo general, este no es el uso previsto para los `tf.sparse.SparseTensor`. Probablemente, le convenga considerar otras técnicas para codificar, tales como por ejemplo, usar un tensor de máscara separado con el que se identifiquen las ubicaciones de valores conocidos o desconocidos. Sin embargo, tenga cuidado al usar esta opción, ya que la mayoría de las operaciones con tensores dispersos tratarán a los valores cero explícitos e implícitos de un modo idéntico.

Tenga en cuenta que algunas operaciones como `tf.sparse.reduce_max` no tratan a los valores faltantes como si fueran cero. Por ejemplo, cuando se ejecuta el bloque de código que se encuentra a continuación, el resultado esperado es `0`. Sin embargo, debido a esta excepción, la salida es `-3`.

In [ ]:
print(tf.sparse.reduce_max(tf.sparse.from_dense([-5, 0, -3])))

Por el contrario, cuando se aplica `tf.math.reduce_max` a un tensor denso, el resultado es 0, tal como se espera.

In [ ]:
print(tf.math.reduce_max([-5, 0, -3]))

## Lecturas y recursos complementarios

- Para conocer más acerca de los tensores consulte la [guía sobre tensores](https://www.tensorflow.org/guide/tensor).
- Lea la [guía sobre tensores irregulares](https://www.tensorflow.org/guide/ragged_tensor) para entender cómo trabajar con tensores irregulares, un tipo de tensor que permite trabajar con datos no uniformes.
- Consulte este modelo de detección de objetos en el [TensorFlow Model Garden](https://github.com/tensorflow/models) que usa tensores disperos en un [decodificador de datos `tf.Example` ](https://github.com/tensorflow/models/blob/9139a7b90112562aec1d7e328593681bd410e1e7/research/object_detection/data_decoders/tf_example_decoder.py).
